In [6]:
using Random, Statistics, LinearAlgebra
using Plots
using BenchmarkTools
using Test

function generate_hopfield_sample(N, α)
    M = round(Int, N*α)
    ξ = rand([-1,1], N, M)
    J = (ξ * ξ') ./ N
    return ξ, J
end

rand_config(N::Int) = rand([-1,1], N)

"""
Flip each spin of `σ` with probability `p`.
"""
function perturb_config(σ::AbstractVector, p)
    @assert 0 <= p <= 1
    σnew = copy(σ)
    for i in 1:length(σnew)
        if rand() < p
            flip!(σnew, i)
        end 
    end
    return σnew
end

function energy(J::AbstractMatrix, σ::AbstractVector)
    return -(σ' * J * σ) / 2
end

function staggered_mag(ξ::AbstractVector, σ::AbstractVector)
    return ξ ⋅ σ / length(σ)
end

function staggered_mags(ξ::AbstractMatrix, σ::AbstractVector)
    return ξ' * σ ./ length(σ)
end

function is_local_minimum(J, σ)
    h = J * σ .- diag(J) .* σ
    return σ == sign.(h) 
end

flip!(σ, i::Int) = σ[i] = -σ[i]

function delta_energy(J, σ, i::Int)
    hi = @views J[:, i] ⋅ σ  - J[i, i] * σ[i]  # this dot product dominates the compute time of the whole simulation
    return 2hi * σ[i]
end

function one_mc_step!(J, σ, i, T)
    flipped = false
    ΔE = delta_energy(J, σ, i)
    # ΔE = randn()
    if ΔE < 0 || rand() < exp(-ΔE / T)
        flip!(σ, i)
        flipped = true
    end
    #if rand() < exp(-ΔE / T)
     #   flip!(σ, i)
      #  flipped = true
    #end
    return flipped
end

function one_mc_sweep!(J, σ, T)
    N = length(σ)
    nflips = 0
    for _ in 1:N
        i = rand(1:N)
        nflips += one_mc_step!(J, σ, i, T)  
    end
    return nflips / N 
end

function run_mcmc(J::AbstractMatrix;  
            σ0::AbstractVector = rand([-1,1], size(J, 1)), 
            T=1e-6,
            nsweeps=100, 
            infotime=0, # Set to 0 for no printing
            earlystop=0, # Stop if fliprate is equal or less than this threshold.
                         # Set to negative for no early stopping
            )
    
    N = size(J, 1)
    @assert length(σ0) == N
    @assert all(∈([-1,1]), σ0)
    σ = copy(σ0)

    function report(t, fliprate)
        E = energy(J, σ) / N
        @info (; t, E, fliprate)
    end

    infotime > 0 && report(0, missing)
    for t in 1:nsweeps
        fliprate = one_mc_sweep!(J, σ, T)
        if infotime > 0 && t % infotime == 0
            report(t, fliprate)
        end
        fliprate <= earlystop && break
    end
    return σ
end

run_mcmc (generic function with 1 method)

In [11]:
# Random.seed!(17)
c = 0
@time begin
    for _ in 1:1
        N = 5000  
        α = 0.1
        p = 0.4 # perturb probability
        ξ, J = generate_hopfield_sample(N, α)
        σ0 = perturb_config(ξ[:,1], p)

        #print(staggered_mag(σ0, ξ[:,1]), "\n")
        σ = run_mcmc(J; σ0, nsweeps=100, T=1e-6)
        #print(staggered_mag(σ, ξ[:,1]), "\n")
        m = staggered_mags(ξ, σ)
        if m[1] >= 0.95
            c +=1
        end
    end
end
#print( staggered_mag(ξ[:, 1], σ), "\n")
print(c)


 10.715879 seconds (19 allocations: 400.696 MiB, 0.50% gc time)
0

In [ ]:
#### BENCHMARK && PROFILING ###############

# σ0 = rand([-1, 1], N)
# @btime run_mcmc(J; σ0, nsweeps=10, earlystop=-1, infotime=0)
## N=100, α = 0.1, on my laptop:
## 38.567 μs (4 allocations: 1.05 KiB) 

# VSCodeServer.@profview run_mcmc(J; σ0, nsweeps=10, earlystop=-1, infotime=0)
## All time spent in dot() for computing hi, so cannot do anything to improve performance

### TESTS BELOW ##############
# TODO move to separate file

@testset "delta_energy" begin
    N = 100
    α = 0.1
    ξ, J = generate_hopfield_sample(N, α)
    
    σ = rand([-1, 1], N)
    i = rand(1:N)
    
    E0 = energy(J, σ)
    flip!(σ, i)
    E1 = energy(J, σ)
    flip!(σ, i)
    @test delta_energy(J, σ, i) ≈ E1 - E0
end

@testset "at high temp accept every moove" begin
    N = 100
    α = 0.1
    ξ, J = generate_hopfield_sample(N, α)
    
    T = Inf
    σ = rand([-1, 1], N)
    fliprate = one_mc_sweep!(J, σ, T)
    @test fliprate == 1

    T = 10
    σ = rand([-1, 1], N)
    fliprate = one_mc_sweep!(J, σ, T)
    @test fliprate > 0.95

    T = 1
    σ = rand([-1, 1], N)
    fliprate = one_mc_sweep!(J, σ, T)
    @test 0.5 < fliprate < 0.9
end

@testset "patterns are fixed points at low alpha" begin
    N = 100
    α = 0.1
    ξ, J = generate_hopfield_sample(N, α)
    @test is_local_minimum(J, ξ[:,1])
    
    σ0 = ξ[:,1]
    σ = run_mcmc(J; σ0, nsweeps=3, infotime=100)
    @test σ == σ0
    @test σ !== σ0
    m = staggered_mags(ξ, σ)
    @test m[1] == 1
end   


@testset "dynamics from random init lands in pattern at low alpha" begin
    # THIS IS TRUE ONLY FOR SMALL N, AT LARGE N IT GETS 
    # TRAPPED BY SPURIOUS MINIMA
    N = 100
    α = 0.1
    ξ, J = generate_hopfield_sample(N, α)
    σ0 = rand([-1,1], N)
    σ = run_mcmc(J; σ0, nsweeps=100, infotime=100)
    
    m0 = staggered_mags(ξ, σ0)
    m = staggered_mags(ξ, σ)
    μ0max = argmax(m0) 
    μmax = argmax(m)
    mmax = m[μmax]
    @test mmax > 0.95
    # @test μ0max == μmax # not generally true
end

@testset "perturb_config" begin
    N = 100
    σ = rand_config(N)
    σnew = perturb_config(σ, 0)
    @test σnew == σ

    for p in [0.1, 0.5, 0.9]
        σnew = perturb_config(σ, p)
        @test 1-2p - 0.1 < staggered_mag(σ, σnew) < 1-2p + 0.1
    end

    σnew = perturb_config(σ, 1)
    @test σnew == -σ
end